In [2]:
## Task 1 Tokenize
import nltk
f=open('C:/Users/65730/TrumpImpeach_Reddit_Comments.txt',encoding = "utf-8")
raw = f.read()

In [3]:
sentences = nltk.sent_tokenize(raw)

In [4]:
sentences[:1]

["['House Passes Trump Impeachment Inquiry, Moving to Public Phase', '>Practically speaking, the resolution outlines the rights and procedures that will guide the process from here on out, including the public presentation of evidence and how Mr. Trump and his legal team will be able to eventually mount a defense.\\n>\\n>But its significance was more profound: After five weeks of private fact-finding, an almost completely unified Democratic caucus signaled that, despite Republican opposition, they now have enough confidence in the severity of the underlying facts about Mr. Trump’s dealings with Ukraine to start making their case for impeachment in public.\\n\\nPaywall workaround:\\n\\nhttp://archive.is/6oLGL', 'What led to Trump’s first meeting on June 20, 2017, with Ukraine’s then-President Petro Poroshenko?"]

In [5]:
words=[nltk.word_tokenize(word)for word in sentences]
words[:1]

[['[',
  "'House",
  'Passes',
  'Trump',
  'Impeachment',
  'Inquiry',
  ',',
  'Moving',
  'to',
  'Public',
  'Phase',
  "'",
  ',',
  "'",
  '>',
  'Practically',
  'speaking',
  ',',
  'the',
  'resolution',
  'outlines',
  'the',
  'rights',
  'and',
  'procedures',
  'that',
  'will',
  'guide',
  'the',
  'process',
  'from',
  'here',
  'on',
  'out',
  ',',
  'including',
  'the',
  'public',
  'presentation',
  'of',
  'evidence',
  'and',
  'how',
  'Mr.',
  'Trump',
  'and',
  'his',
  'legal',
  'team',
  'will',
  'be',
  'able',
  'to',
  'eventually',
  'mount',
  'a',
  'defense.\\n',
  '>',
  '\\n',
  '>',
  'But',
  'its',
  'significance',
  'was',
  'more',
  'profound',
  ':',
  'After',
  'five',
  'weeks',
  'of',
  'private',
  'fact-finding',
  ',',
  'an',
  'almost',
  'completely',
  'unified',
  'Democratic',
  'caucus',
  'signaled',
  'that',
  ',',
  'despite',
  'Republican',
  'opposition',
  ',',
  'they',
  'now',
  'have',
  'enough',
  'confidenc

In [6]:
## Task 2 Frequency distribution
## get all words from all comments and put into a frequency distribution
#   note lowercase, but no stemming or stopwords
word1=nltk.word_tokenize(raw)
all_words = nltk.FreqDist(word1)
# get the 2000 most frequently appearing keywords in the corpus
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]
print(word_features[:50])

[',', '.', 'the', "'", 'to', '``', 'a', 'and', 'of', 'is', 'that', 'in', 'I', '’', ':', 'he', 'it', 'Trump', "'s", 'for', '?', 'they', 'his', 'be', 'this', ')', 'on', 'are', '(', 'with', 's', 'was', 'you', 'have', "''", 'not', '!', 'as', '[', ']', 'will', 'him', '2020', "n't", 'https', '\\n\\n', 'Election', 'all', 'just', 'but']


In [7]:
## Task 3 Sentiment classification 
#Sentiment Classification - Words as Features (BOW or unigram features)  
#load the sentence_polarity corpus and create a list of documents   
from nltk.corpus import sentence_polarity  
import random  
random.seed(30)   
sentences=sentence_polarity.sents()  
documents= [(sent,cat) for cat in sentence_polarity.categories()  
         for sent in sentence_polarity.sents(categories=cat)]  
random.shuffle(documents) #random shuffle documents  

In [8]:
##define the set of words that will be used for features   
##limit to the 2000 most frequent words   
##lowercase words without stemming or removing stopwords   
all_words_list=[word for (sent,cat) in documents for word in sent]  
all_words = nltk.FreqDist(all_words_list)  
word_items = all_words.most_common(2000)  
word_features = [word for (word,freq) in word_items]  

In [9]:
#define the unigram features for each document, using just the words  
#The feature label will be ‘contains(keyword)’ for each keyword (aka word) in the word_features set，the value is whether the word is contained in the document  
def document_features(document,word_features):  
    document_words=set(document)  
    features={}  
    for word in word_features:  
        features['contain({})'.format(word)] = (word in document_words)  
    return features  
#Define the feature sets for the documents.  
featuresets=[(document_features(d,word_features),c) for (d,c) in documents]  
# do a 90/10 split of our approximately 10,000 documents  
word_train_set,word_test_set = featuresets[1000:],featuresets[:1000]  
classifier=nltk.NaiveBayesClassifier.train(word_train_set)  
print(nltk.classify.accuracy(classifier,word_test_set))#0.727  

0.727


In [10]:
#Sentiment Classification - Subjectivity Count features  
#read in the subjectivity words from the subjectivity lexicon  
SLpath = 'C:/Users/65730/subjclueslen1-HLTEMNLP05.tff'  

def readSubjectivity(path):  
    flexicon = open(path, 'r')  
    # initialize an empty dictionary  
    sldict = { }  
    for line in flexicon:  
        fields = line.split()   # default is to split on whitespace  
        # split each field on the '=' and keep the second part as the value  
        strength = fields[0].split("=")[1]  
        word = fields[2].split("=")[1]  
        posTag = fields[3].split("=")[1]  
        stemmed = fields[4].split("=")[1]  
        polarity = fields[5].split("=")[1]  
        if (stemmed == 'y'):  
            isStemmed = True  
        else:  
            isStemmed = False  
        # put a dictionary entry with the word as the keyword  
        #     and a list of the other values  
        sldict[word] = [strength, posTag, isStemmed, polarity]  
    return sldict   

In [11]:
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
#    2 * number of strongly negative words
# positive feature has similar definition
#    not counting neutral words
SL = readSubjectivity(SLpath) 
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    # count variables for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word in SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'weaksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features

SL_featuresets = [(SL_features(d, word_features, SL), c) for (d, c) in documents]

In [12]:
# show just the two sentiment lexicon features in document 0
print(SL_featuresets[0][0]['positivecount'])
print(SL_featuresets[0][0]['negativecount'])
SL_featuresets[0][1]

2
0


'pos'

In [13]:
# retrain the classifier using these features
SL_train_set, SL_test_set = SL_featuresets[1000:], SL_featuresets[:1000]
SL_classifier = nltk.NaiveBayesClassifier.train(SL_train_set)
nltk.classify.accuracy(SL_classifier, SL_test_set)

0.762

In [14]:
###task 4 Negation features 

# Negation words "not", "never" and "no"
# Not can appear in contractions of the form "doesn't"
for sent in list(sentences)[:50]:
   for word in sent:
     if (word.endswith("n't")):
       print(sent)

# this list of negation words includes some "approximate negators" like hardly and rarely
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

# One strategy with negation words is to negate the word following the negation word
#   other strategies negate all words up to the next punctuation
# Strategy is to go through the document words in order adding the word features,
#   but if the word follows a negation words, change the feature to negated word
# Start the feature set with all 2000 word features and 2000 Not word features set to false
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features

# define the feature sets
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]

not_train_set, not_test_set = NOT_featuresets[1000:], NOT_featuresets[:1000]
NOT_classifier  = nltk.NaiveBayesClassifier.train(not_train_set)
print(nltk.classify.accuracy(NOT_classifier, not_test_set)) 

['there', 'is', 'a', 'difference', 'between', 'movies', 'with', 'the', 'courage', 'to', 'go', 'over', 'the', 'top', 'and', 'movies', 'that', "don't", 'care', 'about', 'being', 'stupid']
['a', 'farce', 'of', 'a', 'parody', 'of', 'a', 'comedy', 'of', 'a', 'premise', ',', 'it', "isn't", 'a', 'comparison', 'to', 'reality', 'so', 'much', 'as', 'it', 'is', 'a', 'commentary', 'about', 'our', 'knowledge', 'of', 'films', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['i', "didn't", 'laugh', '.', 'i', "didn't", 'smile', '.', 'i', 'survived', '.']
['most', 'of', 'the', 'problems', 'with', 'the', 'film', "don't", 'derive', 'from', 'the', 'screenplay', ',', 'but', 'rather', 'the', 'mediocre', 'performances', 'by', 'most', 'of', 'the', 'actors', 'involved']
['the', 'lack', 'of', 'naturalness', 'makes', 'everything', 'seem', 'self-consciously', 'poetic', 'and', 'forced', '.', '.', '.', "it's", 'a', 'pity', 'that', "[nelson's]", 'achievement', "doesn't", 'match'

In [15]:
# Remove stop words and re-run SL_features  
#remove stop words in word features using negation filter  
stopwords = nltk.corpus.stopwords.words('english')  
newstopwords = [word for word in stopwords if word not in negationwords]  
new_all_words_list=[word for word in all_words_list if word not in newstopwords]  
#get new word features of length 2000 after the stopwords are removed  
new_all_words = nltk.FreqDist(new_all_words_list)  
new_word_items = new_all_words.most_common(2000)  
new_word_features = [word for (word,count) in new_word_items]  
#re run SL_features  
def newSL_features(document, new_word_features, SL):  
    document_words = set(document)  
    features = {}  
    for word in new_word_features:  
        features['contains({})'.format(word)] = (word in document_words)  
    # count variables for the 4 classes of subjectivity  
    weakPos = 0  
    strongPos = 0  
    weakNeg = 0  
    strongNeg = 0  
    for word in document_words:  
        if word in SL:  
            strength, posTag, isStemmed, polarity = SL[word]  
            if strength == 'weaksubj' and polarity == 'positive':  
                weakPos += 1  
            if strength == 'strongsubj' and polarity == 'positive':  
                strongPos += 1  
            if strength == 'weaksubj' and polarity == 'negative':  
                weakNeg += 1  
            if strength == 'strongsubj' and polarity == 'negative':  
                strongNeg += 1  
            features['positivecount'] = weakPos + (2 * strongPos)  
            features['negativecount'] = weakNeg + (2 * strongNeg)  
    return features  
newSL_featuresets = [(newSL_features(d, new_word_features, SL), c) for (d, c) in documents]  
newSL_train_set, newSL_test_set = newSL_featuresets[1000:],newSL_featuresets[:1000]  
newSLclassifier = nltk.NaiveBayesClassifier.train(newSL_train_set)  
print(nltk.classify.accuracy(newSLclassifier, newSL_test_set)) #0.749  

0.749


In [16]:
#Subjectivity Count features- SL_classifier precision, recall and F-measure  
# evaluation measures showing performance of negation  
from nltk.metrics import *  
reflist = []  
testlist = []  
for (features, label) in SL_test_set:  
    reflist.append(label)  
    testlist.append(NOT_classifier.classify(features))  
# define and print confusion matrix  
cm = ConfusionMatrix(reflist, testlist)  
print(cm)  
refpos = set()  
refneg = set()  
testpos = set()  
testneg = set()  


    |   n   p |
    |   e   o |
    |   g   s |
----+---------+
neg |<366>128 |
pos | 147<359>|
----+---------+
(row = reference; col = test)



In [17]:
for i, label in enumerate(reflist):  
    if label == 'pos': refpos.add(i)  
    if label == 'neg': refneg.add(i)  
  
for i, label in enumerate(testlist):  
    if label == 'pos': testpos.add(i)  
    if label == 'neg': testneg.add(i)  
# compute precision, recall and F-measure for each label  
def printmeasures(label, refset, testset):  
    print(label, 'precision:', precision(refset, testset))  
    print(label, 'recall:', recall(refset, testset))  
    print(label, 'F-measure:', f_measure(refset, testset))  

In [18]:
printmeasures('pos', refpos, testpos)  

pos precision: 0.7371663244353183
pos recall: 0.7094861660079052
pos F-measure: 0.7230614300100705


In [19]:
#Predict reddit comments with SL_Classifier   
#Get the predict results after applying classifier  
pos_list=[]  
neg_list=[]  
for comments in words:  
    if(SL_classifier.classify(SL_features(comments,word_features,SL))=='pos'):  
        #use string.join() combine list  
       pos_list.append(" ".join(comments))  
    if(SL_classifier.classify(SL_features(comments,word_features,SL))=='neg'):  
       neg_list.append(" ".join(comments))  
  
print(len(pos_list))  
print(len(neg_list))  


2875
9202


In [21]:
import pandas as pd  
df_pos=pd.DataFrame({'$Positive':pos_list})  
df_neg=pd.DataFrame({'$Negative':neg_list})  
df_pos.to_csv('CommentPostive.csv',index=False)  
df_neg.to_csv('CommentNegative.csv',index=False) 